# Logit Regression Model for Features Importance: `/r/PC` and `/r/PCM`
- This notebook is used to create the processed data for 3.1-notebook.

In [1]:
# Packages
from matplotlib import rc
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import sklearn.preprocessing
import sklearn.utils
import statsmodels.api as sm
import statsmodels.formula.api as smf
from tqdm import tqdm
import argparse
import logging
import sys

sys.path += ['../']

from config import processed_data_path, raw_data_path, figure_path

# Parameters
DATA_PATH = raw_data_path
OUTPUT_PATH = processed_data_path
FIGURES_PATH = figure_path

SUBREDDITS = {
    'PoliticalCompass': 'PC',
    'PoliticalCompassMemes': 'PCM'}

POPULARITY_BIN = ['s1', 's2', 's3', 's4']

FLAIRS2SOCIAL = {
    'AuthLeft': 'Auth',
    'AuthCenter': 'Auth',
    'AuthRight': 'Auth',
    'Left': 'Centrist_social',#
    'Centrist': 'Centrist_social',#
    'Right': 'Centrist_social',#
    'LibLeft': 'Lib',
    'LibCenter': 'Lib',
    'LibRight': 'Lib'
}
FLAIRS2ECONOMIC = {
    'AuthLeft': 'Left',
    'Left': 'Left',
    'LibLeft': 'Left',
    'AuthCenter': 'Centrist_economic',#
    'Centrist': 'Centrist_economic',#
    'LibCenter': 'Centrist_economic',#
    'LibRight': 'Right',
    'Right': 'Right',
    'AuthRight': 'Right'
}
SOCIO_DEM2CLASSES = {
    'age': ['Young', 'Old'],    
    'gender': ['Male', 'Female'],   
    'affluence': ['Poor', 'Rich']
}
SOCIO_DEM_CLASSES = [x for pair in SOCIO_DEM2CLASSES.values() for x in pair]
N_SOCIO = len(SOCIO_DEM_CLASSES)

IDEOLOGIES2CLASSES = {
    'economic': ['Left','Right'],
    'social': ['Lib', 'Auth']
}

IDEOLOGIES_CLASSES = [x for pair in IDEOLOGIES2CLASSES.values() for x in pair]
N_IDEOLOGIES = len(IDEOLOGIES_CLASSES)

CLASSES = np.concatenate([SOCIO_DEM_CLASSES,
                          IDEOLOGIES_CLASSES,
                          POPULARITY_BIN
                         ])
N = len(CLASSES)

# Functions
def ideologic_features(edges: pd.DataFrame, users: pd.DataFrame) -> pd.DataFrame:
    # edges: child, parent
    # users: author, flair
    if ('child' in set(edges.keys())) & ('parent' in set(edges.keys())) & \
    ('author' in set(users.keys())) & ('flair' in set(users.keys())):
        df = (edges
              .join(users.set_index('author'), on='child')
              .rename(columns={'flair': 'social_child'})
              .join(users.set_index('author'), on='child')
              .rename(columns={'flair': 'economic_child'})
              .join(users.set_index('author'), on='parent')
              .rename(columns={'flair': 'social_parent'})
              .join(users.set_index('author'), on='parent')
              .rename(columns={'flair': 'economic_parent'})
             )

        df['social_child'] = df['social_child'].map(FLAIRS2SOCIAL.get).values
        df['economic_child'] = df['economic_child'].map(FLAIRS2ECONOMIC.get).values

        df['social_parent'] = df['social_parent'].map(FLAIRS2SOCIAL.get).values
        df['economic_parent'] = df['economic_parent'].map(FLAIRS2ECONOMIC.get).values
        
        return df
        # (return) edges: child, parent, Social_child, Economic_social, Social_parent, Economic_parent
    else:
        return pd.DataFrame({})

def quantile_normalize(data):
    return sklearn.preprocessing.quantile_transform(
            data.values.reshape(-1, 1), copy=True)

def subsample_neg_pairs(graph_df):
    """
    Random Network (Null Model) as a directed, weighted, random network (RN).
    This Network is obtained by reshuffling links of the original network while preserving
    the in- and out-strength (weighted degree) of each node.
    Args:
      graph_df: A Pandas DataFrame of edges.
      !!! Important: graph_df should contain ONLY `child` & `parent` columns
    Returns:
      A Pandas DataFrame of negative edges.
    """
    edges_set = set(map(tuple, graph_df.values)) # Set of all edges in the graph
    non_edges = [] # Empty list to store the negative edges
    pbar = tqdm(total=len(graph_df))# Progress bar
    while len(non_edges) < len(graph_df):
        sample_size = min(2 ** 17, len(graph_df) - len(non_edges)) # 2 ** 17 overflow
        # Randomly sample `sample_size` values from each column in the graph_df DataFrame
        column_samples = [np.random.choice(graph_df[c], sample_size) for c in graph_df.columns]
        # Create a list of tuples representing the negative edges
        new_non_edges = [sample for sample in zip(*column_samples) if sample not in edges_set]
        # Add the new negative edges to the list of negative edges
        non_edges += new_non_edges
        # Update the progress bar
        pbar.update(len(new_non_edges))
    
    pbar.close()# Close progress bar
    return pd.DataFrame(non_edges, columns=graph_df.columns)

### 0 Load data


In [2]:
# Read Data
# Users and edges of interaction networks, users socio-demographic features
users_df = {}
edges_df = {}
socio_demographic_features = {}
confounding_features = {}

for S in SUBREDDITS:
    
    # Users with single flair
        # author, flair
    users_df[S] = pd.read_csv(OUTPUT_PATH + f"single_flair_anonymized_users_{SUBREDDITS[S]}.csv")
    print(f"{S}:")
    print(f"\t users loaded (unique users: {len(users_df[S])}) ✓")
    
    # Edges from Interaction Network
        # child, parent
    edges_df[S] = pd.read_csv(DATA_PATH + f"edges_anonymized_{SUBREDDITS[S]}.csv")
    
    edges_df[S] = edges_df[S][['child', 'parent']]
    print(f"\t edges loaded (unique users: {len(set(edges_df[S]['child'].unique()).union(set(edges_df[S]['parent'].unique())))}) ✓")
    
    # Users scores for socio-demographic features
        # 'user', 'age', 'gender', 'affluence', 'social', 'economic'
    socio_demographic_features[S] = (pd.read_csv(DATA_PATH + f"socio_demographics_anonymized_{SUBREDDITS[S]}.csv"))
    print(f"\t socio-demographic features loaded (unique users: {len(set(socio_demographic_features[S]['user'].unique()))}) ✓")
    # Remove not usefull columns (partisan, flair)
    socio_demographic_features[S] = (socio_demographic_features[S].drop(columns=['partisan', 'flair']))
    # Modify values for social/economic Centrist: Centrist -> Centrist_social/Centrist_economic
    for axis in ['social', 'economic']:
        socio_demographic_features[S][axis] = (socio_demographic_features[S]
                                               .apply(lambda x: x[axis] + f'_{axis}'
                                                      if x[axis] == 'Centrist' else x[axis],
                                                      axis=1
                                                     )
                                              )
    # Bool values for socio-demographic
    for feature, (class_low, class_hi) in SOCIO_DEM2CLASSES.items():
            socio_demographic_features[S][class_low] = socio_demographic_features[S][feature] <= 0.25
            socio_demographic_features[S][class_hi] = socio_demographic_features[S][feature] >= 0.75
            del socio_demographic_features[S][feature]
    print(f"\t socio-demographic features modified ✓")
    
    # Confounding features:
        # 'author', 'flair', 'num_comm', 'avg_score', 'num_comm_pos', 'frac_pos'
    confounding_features[S] = pd.read_csv(OUTPUT_PATH + f"popularity_metrics_anonymized_{SUBREDDITS[S]}.csv")
    print(f"\t confounding features loaded ✓")

    # Select comments number, average score and positive score comments number
    confounding_features[S] = (users_df[S].join(confounding_features[S][['author', 'avg_score']].set_index('author'), on='author'))
    print(f"\t confounding features selected ✓")
    
    #confounding_features[S]['frac_pos'] = confounding_features[S]['num_comm_pos'] / confounding_features[S]['num_comm']
    #confounding_features[S] = confounding_features[S].dropna()
    confounding_features[S]['avg_score_q'] = quantile_normalize(confounding_features[S]['avg_score'])
    confounding_features[S]['popularity'] = pd.qcut(
        confounding_features[S]['avg_score_q'],
        q=len(POPULARITY_BIN),
        labels=POPULARITY_BIN, retbins=True)[0]
    confounding_features[S] = confounding_features[S].dropna()
    print(f"\t users popularity classified ✓")

PoliticalCompass:
	 users loaded (unique users: 22503) ✓
	 edges loaded (unique users: 19124) ✓
	 socio-demographic features loaded (unique users: 18255) ✓
	 socio-demographic features modified ✓
	 confounding features loaded ✓
	 confounding features selected ✓
	 users popularity classified ✓
PoliticalCompassMemes:
	 users loaded (unique users: 258428) ✓
	 edges loaded (unique users: 223304) ✓
	 socio-demographic features loaded (unique users: 251344) ✓
	 socio-demographic features modified ✓
	 confounding features loaded ✓
	 confounding features selected ✓
	 users popularity classified ✓


In [3]:
def create_features(df):
    return create_political_compass_features(df) + create_socio_attrib_features(df) + create_popularity_features(df)

def create_popularity_features(df):
    return [("Target_is_popular", df['s4_parent']), ("Target_is_not_popular", df['s1_parent'])]

def create_socio_attrib_features(df):
    import itertools
    socio_attrib_combinations = list(itertools.combinations(SOCIO_DEM_CLASSES, 2)) + [(a, a) for a in SOCIO_DEM_CLASSES]
    return [(
                f"Symmetric_{a1}_{a2}", (df[f'{a1}_child'] & df[f'{a2}_parent']) | (df[f'{a2}_child'] & df[f'{a1}_parent'])                           
            ) 
            for a1, a2 in socio_attrib_combinations
    ]

def create_political_compass_features(df):

    return [
        (
            "Homo_Economic", (
                            (df['Left_child'] & df['Left_parent']) | 
                            (df['Right_child'] & df['Right_parent'])
                            )
        ),
        (
            "Hete_Economic", (
                            (df['Left_child'] & df['Right_parent']) | 
                            (df['Right_child'] & df['Left_parent'])
                            )
        ),
        (
            "Hete_Economic_Source_Left", (
                            (df['Left_child'] & df['Right_parent'])
                            )
        ),
        (
            "Homo_Social", (
                            (df['Lib_child'] & df['Lib_parent']) | 
                            (df['Auth_child'] & df['Auth_parent'])
                            )
        ),
        (
            "Hete_Social", (
                            (df['Lib_child'] & df['Auth_parent']) | 
                            (df['Auth_child'] & df['Lib_parent'])
                            )
        ),
        (
            "Hete_Social_Source_Lib", (
                            (df['Lib_child'] & df['Auth_parent'])
                            )
        ),
    ]

# Build X and y

In [4]:
edges = {}
non_edges = {}
regression_df = {}
regression_variables = {}
variable_names = {}
X = {}
y = {}
logreg = {}
logreg_res = {}

for S in SUBREDDITS:
    np.random.seed(44)
    
    edges[S] = edges_df[S][(edges_df[S]['child'].isin(set(socio_demographic_features[S]['user'].unique()))) & 
                           (edges_df[S]['parent'].isin(set(socio_demographic_features[S]['user'].unique())))
                          ]
    non_edges[S] = subsample_neg_pairs(edges[S])

    edges[S]['is_link'] = 1
    non_edges[S]['is_link'] = 0
    print(f"{S}:")
    print(f"\t sampled negative edges ✓")
    
    regression_df[S] = ideologic_features(pd.concat([edges[S], non_edges[S]]), users_df[S][['author', 'flair']])
        #child, parent, is_link, Left_child, Left_parent, Centrist_economic_child, Centrist_economic_parent,
        #Right_child, Right_parent, Lib_child, Lib_parent, Centrist_social_child, Centrist_social_parent,
        #Auth_child, Auth_parent
    for feature, classes in IDEOLOGIES2CLASSES.items():
        for c in classes:
            regression_df[S][f'{c}_child'] = regression_df[S][f'{feature}_child'] == c
            regression_df[S][f'{c}_parent'] = regression_df[S][f'{feature}_parent'] == c
        del regression_df[S][f'{feature}_child']
        del regression_df[S][f'{feature}_parent']
    print(f"\t get ideologies for child, parent ✓")
    
    regression_df[S] = (regression_df[S]
                        # Popularity features (child, parent)
                         .merge(confounding_features[S][['author', 'popularity']].set_index('author'),
                                right_on='author', left_on='child')
                        .merge(confounding_features[S][['author', 'popularity']].set_index('author'),
                               right_on='author', left_on='parent', suffixes=('_child', '_parent'))
                        # Socio-demographic features (child, parent)
                        .merge((socio_demographic_features[S]
                                [['user', 'Young', 'Old', 'Male', 'Female', 'Poor', 'Rich']]
                                .set_index('user')), left_on='child', right_on='user')
                        .merge((socio_demographic_features[S]
                                [['user', 'Young', 'Old', 'Male', 'Female', 'Poor', 'Rich']]
                                .set_index('user')),
                               left_on='parent', right_on='user', suffixes=('_child', '_parent'))
                       )
    print(f"\t get confounging & socio-demographic features ✓")
    
    for popularity in POPULARITY_BIN:
        regression_df[S][f'{popularity}_child'] = regression_df[S]['popularity_child'] == popularity
        regression_df[S][f'{popularity}_parent'] = regression_df[S]['popularity_parent'] == popularity
    del regression_df[S]['popularity_child']
    del regression_df[S]['popularity_parent']
    print(f"\t get popularity classes for child, parent")
        
    regression_df[S] = regression_df[S][['is_link', # Y
                                         # X
                                             # ideologies
                                         'Left_child', 'Left_parent', #'Centrist_economic_child',
                                         'Right_child', 'Right_parent', 'Lib_child', # 'Centrist_economic_parent', 
                                         'Lib_parent', #'Centrist_social_child', #'Centrist_social_parent',
                                         'Auth_child', 'Auth_parent',
                                             # socio-demographic
                                         'Young_child', 'Old_child', 'Male_child', 'Female_child', 'Poor_child',
                                         'Rich_child', 'Young_parent', 'Old_parent', 'Male_parent',
                                         'Female_parent', 'Poor_parent', 'Rich_parent',
                                             # popularity
                                         's1_child', 's1_parent', 's2_child','s2_parent',
                                         's3_child', 's3_parent', 's4_child', 's4_parent'
                                        ]]
    print(f"\t get Y (is_link: 1), X (ideologies: 12, socio-dem: 12, confounding: 8) ✓")
    
    regression_df[S] = sklearn.utils.shuffle(regression_df[S])
    print(f"\t shuffled edges ✓")    
    
    regression_variables[S] = create_features(regression_df[S])
    variable_names[S] = [name for name, value in regression_variables[S]]
    print(f"\t get variables names ✓")

    # Compute X
    batch_size = 1

    # Calculate the total number of rows that will be in the final array
    total_rows = len(regression_variables[S])

    # Calculate the number of columns based on one of the arrays
    num_columns = len(regression_variables[S][0][1])

    # Preallocate the array
    X[S] = np.empty((total_rows, num_columns), dtype=np.int8)

    # Fill the preallocated array in batches
    for i in tqdm(range(0, total_rows, batch_size)):
        batch = regression_variables[S][i:i + batch_size]
        stacked = np.vstack([value for name, value in batch]).astype(np.int8)
        X[S][i:i + batch_size] = stacked  # Fill in place

    X[S] = X[S].T

    print(f"\t get X ✓")

    y[S] = np.array(regression_df[S].is_link.values)
    print(f"\t get Y ✓")

    logreg[S] = sm.Logit(y[S], sm.add_constant(X[S], has_constant='add'), missing='raise')
    logreg_res[S] = logreg[S].fit(maxiter=1000, disp=True, method='lbfgs') #'lbfgs'
    
    # Extract the relevant data
    coefficients = logreg_res[S].params[1:]  # skip the constant
    conf_int = logreg_res[S].conf_int(alpha=0.05)[1:]  # 95% CI
    pvalues = logreg_res[S].pvalues[1:]

    # Create a DataFrame
    data = pd.DataFrame({
        'feature_name': variable_names[S],
        'coefficient': coefficients,
        'pvalue': pvalues,
        'conf_int_lower': conf_int[:, 0],
        'conf_int_upper': conf_int[:, 1]
    })

    data.to_csv(OUTPUT_PATH + f"features_importance_{SUBREDDITS[S]}.csv", index=False)

100%|██████████| 261078/261078 [00:00<00:00, 2999059.23it/s]
/tmp/ipykernel_1232109/3602297826.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges[S]['is_link'] = 1


PoliticalCompass:
	 sampled negative edges ✓
	 get ideologies for child, parent ✓
	 get confounging & socio-demographic features ✓
	 get popularity classes for child, parent
	 get Y (is_link: 1), X (ideologies: 12, socio-dem: 12, confounding: 8) ✓
	 shuffled edges ✓
	 get variables names ✓


100%|██████████| 29/29 [00:00<00:00, 4130.92it/s]

	 get X ✓
	 get Y ✓


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           30     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  5.46806D-03

At iterate    1    f=  6.92188D-01    |proj g|=  1.18237D-03

At iterate    2    f=  6.92091D-01    |proj g|=  9.06011D-04

At iterate    3    f=  6.92005D-01    |proj g|=  1.10593D-03

At iterate    4    f=  6.91998D-01    |proj g|=  6.18178D-04


...

PoliticalCompassMemes:
	 sampled negative edges ✓
	 get ideologies for child, parent ✓
	 get confounging & socio-demographic features ✓
	 get popularity classes for child, parent
	 get Y (is_link: 1), X (ideologies: 12, socio-dem: 12, confounding: 8) ✓
	 shuffled edges ✓
	 get variables names ✓


100%|██████████| 29/29 [00:00<00:00, 123.04it/s]


	 get X ✓
	 get Y ✓
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           30     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  8.04355D-03


 This problem is unconstrained.



At iterate    1    f=  6.91243D-01    |proj g|=  6.09800D-03

At iterate    2    f=  6.91151D-01    |proj g|=  1.09544D-03

At iterate    3    f=  6.91063D-01    |proj g|=  9.73142D-04

At iterate    4    f=  6.90994D-01    |proj g|=  4.34950D-04

At iterate    5    f=  6.90969D-01    |proj g|=  4.21634D-04

At iterate    6    f=  6.90961D-01    |proj g|=  4.47200D-04

At iterate    7    f=  6.90953D-01    |proj g|=  1.00905D-04

At iterate    8    f=  6.90952D-01    |proj g|=  4.12017D-05

At iterate    9    f=  6.90952D-01    |proj g|=  3.98874D-05

At iterate   10    f=  6.90951D-01    |proj g|=  2.25501D-05

At iterate   11    f=  6.90951D-01    |proj g|=  1.78145D-05

At iterate   12    f=  6.90951D-01    |proj g|=  1.47698D-05

At iterate   13    f=  6.90951D-01    |proj g|=  2.08871D-05

At iterate   14    f=  6.90951D-01    |proj g|=  2.19786D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments 